In [ ]:
import pandas as pd
import os
from glob import glob

# Finne data på lagring

### Linuxstammene

In [ ]:
# "Linux-forkortelser" ligger i environment-variablene
FELLES = os.environ['FELLES']
FELLES

In [ ]:
# glob er fin
sorted(glob(FELLES + "/*"))[:5]

In [ ]:
[x for x in sorted(glob(FELLES + "/*")) if 'AKU' in x]

In [ ]:
# evt. listdir
mock_data = FELLES + '/kurs_python_nybegynner/data/mock_sysselsatte'
os.listdir(mock_data)[:5]

In [ ]:
comp = pd.read_csv(mock_data + "/companies_2019_10000.csv")
comp

### X-disk
Hakket mer komplisert enn henting fra linux-stammene.\
Krever også tilganger

In [ ]:
# Denne tar seg av evt. innlogging om det trengs
from xdisk import xdisk
xdisk_path = xdisk()
xdisk_path

In [ ]:
mock_data_xdisk = sorted(glob(xdisk_path + "/A300/360/Brukere/cfc/mock_data/*"))
pops_paths = [x for x in mock_data_xdisk if 'population' in x]
pops_paths

In [ ]:
pop = pd.read_csv(pops_paths[-1])
pop

### Oracle
Krever eget database-passord og **bruker med tilgang** til Oracle-databasen DB1P og tabellene de spørres etter.\
Spørringen under fungerer dermed mest sannsynlig ikke for deg, uten alterasjon til de tabellene du har rettigheter til.\
Funksjonen er satt opp til å spørre om passord hver gang. Om du ikke har lagt det inn i en .env eller environment-variablene til miljøet som "DB1P_PASSWORD" \
Om du legger det i en .env, så settes det noen krav til hvordan passordet kan lagres, det må ikke følge med kodebasen til github feks.

In [ ]:
from db1p import query_db1p

In [ ]:
sporring = """
    SELECT TAB.KOMM_16, TAB.FODT_AAR
    FROM nudb_adm.TAB_UTD_PERSON TAB
    FETCH FIRST 20 ROWS ONLY
"""

In [ ]:
komm_fodt = query_db1p(sporring)
komm_fodt

# Flate filtyper - pandas.read_xxx()

### Parquet
Filformatet som bruke på Dapla, inneholder innebygd datatype-info, kan importeres kolonnevis

In [ ]:
kirkedata = pd.read_parquet("/ssb/bruker/felles/kurs_python_nybegynner/data/kirkedata.parquet.gzip")
kirkedata.info()

### DAT
Flate tekstfiler med satte bredder

In [ ]:
input_sas = """
 INPUT @01 ident    $CHAR15.
       @16 kjonn    $CHAR1.
       @17 kommune  $CHAR4.
       @21 famtype  $CHAR1.
       @22 statsb   $CHAR3.
       @25 foppdato YYMMDD8.
       @33 antbu17       2.
       @35 kapinnt       8.
       @43 samlinnt      8.
       @51 vekt      NUMX6.2
       ;
"""

In [ ]:
# Last inn dat
headers = ['ident', 'kjonn', 'kommune', 'famtype', 'statsb', 'foppdato', 'antbu17', 'kapinnt', 'samlinnt', 'vekt']
prockurs = pd.read_fwf("/ssb/bruker/felles/saskurs/procs/data/prockurs.dat",
                        widths = [15,1,4,1,3,8,2,8,8,6],
                        names = headers,
                        converters={h:str for h in headers}
                       )
prockurs

### Bonus: Parse input-setningen fra SAS for bredder og navn

In [ ]:
# lengder fra input
in_clean = [x.strip() for x in input_sas.replace('\n','')\
                                .replace(':', '')\
                                .replace('INPUT', '')\
                                .split('@')]
in_clean = dict([' '.join(x.split()).split(' ')[1:3] for x in in_clean if x])
in_clean = {k: ''.join([c for c in v if c in '1234567890.']) for k,v in in_clean.items()}
in_clean = {k: int(v.split('.')[0]) for k,v in in_clean.items()}

In [ ]:
# Last inn dat
prockurs = pd.read_fwf("/ssb/bruker/felles/saskurs/procs/data/prockurs.dat",
                        widths = in_clean.values(),
                        names = in_clean.keys(),
                        converters={h:str for h in in_clean.keys()}
                       )

In [ ]:
# Sett datatyper
prockurs[['kapinnt', 'samlinnt']] = prockurs[['kapinnt', 'samlinnt']].astype('Int64')
prockurs['vekt'] = prockurs['vekt'].str.replace(',','.').astype('float')
prockurs.info()

### Excel
Vær bevisst på at hvert "ark" kanskje må importeres for seg.

In [ ]:
medie_tabell = pd.read_excel(f"{xdisk_path}/A300/360/Brukere/cfc/mediebruk/Tabeller_medie_2020_carl_temp_singles.xlsx",
                            sheet_name = "Tabell39" ,
                            header = 3)
medie_tabell

##### Bonus: Rydd opp til multicolumn

In [ ]:
# Kjønn, alder, utdanning
medie_tabell = medie_tabell.dropna(subset=['Mean', 'Mean.1'])
medie_tabell

In [ ]:
medie_tabell = medie_tabell.rename({'Unnamed: 0':'Boklesere'}, axis = 1)
medie_tabell = medie_tabell.set_index('Boklesere')

In [ ]:
medie_tabell = medie_tabell.astype('Int64')
medie_tabell

In [ ]:
lengths = {'Kjønn' : 3, 'Alder' : 6, 'Utdanning' : 5}
index1 = [item for sublist in [[k]*v for k,v in lengths.items()] for item in sublist]
index12 = list(zip(index1, medie_tabell.index))
index = pd.MultiIndex.from_tuples(index12, names = ['Hovedgruppe', 'Undergruppe'])
medie_tabell.index = index
medie_tabell

In [ ]:
medie_tabell.loc['Kjønn']

In [ ]:
medie_tabell.loc['Kjønn','Menn']['N']

# sas7bdat
Anbefalt innlasting er litt kompleks for å gi ekstra hastighet.\
Vi lar her egentlig sas-serverene ta seg av selve fil-lesingen...\
Finne en metode for automatisk innlogging?

In [ ]:
from saspy_session import saspy_df_from_path, set_password

In [ ]:
help(set_password)

In [ ]:
set_password('{SAS004}B...')

In [ ]:
# Åpne fil til pandas
industri2011 = saspy_df_from_path('/ssb/bruker/felles/saskurs/data/industri_2011.sas7bdat')
industri2011.info()

# Data fra APIer

### Data fra statbanken

In [ ]:
from statbankpy import apidata

In [ ]:
trosamfunn = apidata('05300', include_id = True)
trosamfunn

### Tabell fra KLASS (kommunenummer og navn)

In [ ]:
from klass import klass_df
# Alle kommuner som har eksistert mellom 1970 og 2050, hvor vi beholder de sist valide, unike kodene
kommuneinndeling = klass_df('https://data.ssb.no/api/klass/v1/classifications/131/codes.json?from=1970-01-01&to=2050-01-01', 'codes')
kommuneinndeling = kommuneinndeling.sort_values('validToInRequestedRange', ascending=False).drop_duplicates(subset=['code']).sort_values('code')
kommuneinndeling = dict(zip(kommuneinndeling['code'], kommuneinndeling['name']))
#kommuneinndeling

### Bonus: Tilbakefør data via korrespondansetabell fra KLASS

In [ ]:
livssyn_query = {
  "query": [
    {"code": "Region",
      "selection": {
        "filter": "vs:Fylker",
        "values": ["30","01","02","06","03","34","04","05","38","07","08","42","09","10",
                   "11","46","12","13","14","15","50","16","17","18","54","19","20","21",
                    "22","23","25","26","88","99"]
      }
    },
    {
      "code": "ReligionLivs",
      "selection": {
        "filter": "item",
        "values": [
          "900"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}

In [ ]:
livssyn_fylker = apidata('08531', livssyn_query, include_id=True)
livssyn_fylker[livssyn_fylker['region'].str.lower().str.contains('trøn')]

In [ ]:
fylkeskorrespondanse_2019 = klass_df("https://data.ssb.no/api/klass/v1/correspondencetables/452", 'correspondenceMaps')
fylkeskorrespondanse_2018 = klass_df("https://data.ssb.no/api/klass/v1/correspondencetables/588", 'correspondenceMaps')
fylkeskorrespondanse = pd.concat([fylkeskorrespondanse_2019, fylkeskorrespondanse_2018])
fylkeskorrespondanse

In [ ]:
from klass import korrespondanse_summer
# Check mutability?
livssyn_fylker_kombinert = korrespondanse_summer(livssyn_fylker, fylkeskorrespondanse, 'Region', ['value'])

In [ ]:
# Slett de fra datasettet, som ligger i korrespondansen
livssyn_fylker_kombinert = livssyn_fylker_kombinert[livssyn_fylker_kombinert['Region'].isin(fylkeskorrespondanse['sourceCode'])]

In [ ]:
# Skal ha endret seg til å ha verdier i trøndelag før sammenslåingen, og Sør-Trøndelag og Nord-Trøndelag skal være borte
livssyn_fylker_kombinert[livssyn_fylker_kombinert['region'].str.lower().str.contains('trøn')]

# Bonus: enkel linjegraf med plotly

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(livssyn_fylker_kombinert, x="år", y="value", color='region', 
              title=f'Antall medlemmer i Livssyn etter fylke. {min(livssyn_fylker_kombinert["år"])}-{max(livssyn_fylker_kombinert["år"])}')
fig.show()